In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.grammar import PCFG
from nltk.parse import InsideChartParser

In [5]:
def get_Grammar():
    print("Once done with the rule, type \"done\"")
    try:
        return PCFG.fromstring("\n".join(iter(lambda: input("Enter the Rule:").strip(),"done")))
    except ValueError as e:
        print("Error in Grammar Rule: ",e)
def parse_Tree_Construct():
    pcfg=get_Grammar()
    parser=InsideChartParser(pcfg)
    sentence=input("Enter the Sentence:").strip().split()
    parse=list(parser.parse(sentence))
    """if not parse: 
        print("No Parse Tree Found")
        return"""
    parse_data=[(str(tree),tree.prob()) for tree in parse]
    df=pd.DataFrame(parse_data,columns=["Parse Tree","Probability"])
    print("\nMean Probability of Parse Tree:",np.mean(df["Probability"]))
    print("\nParse Tree Result:",df.head())
    for tree in parse: tree.pretty_print()
    return sentence,pcfg

sentence, cnf_grammar=parse_Tree_Construct()

Once done with the rule, type "done"


Enter the Rule:  S -> NP VP [1.0]
Enter the Rule: PP -> P NP [1.0]
Enter the Rule: VP -> V NP [0.7]
Enter the Rule: VP -> VP PP [0.3]
Enter the Rule:  NP -> NP PP [0.4]
Enter the Rule: NP -> 'Astronomers' [0.1]
Enter the Rule: NP -> 'ears' [0.18]
Enter the Rule: NP -> 'saw' [0.04]
Enter the Rule: NP -> 'stars' [0.18]
Enter the Rule: NP -> 'telescope' [0.1]
Enter the Rule: P -> 'with' [1.0]
Enter the Rule: V -> 'saw' [1.0]
Enter the Rule: done
Enter the Sentence: Astronomers saw stars with ears



Mean Probability of Parse Tree: 0.0007938

Parse Tree Result:                                           Parse Tree  Probability
0  (S\n  (NP Astronomers)\n  (VP (V saw) (NP (NP ...     0.000907
1  (S\n  (NP Astronomers)\n  (VP (VP (V saw) (NP ...     0.000680
                  S                    
      ____________|____                 
     |                 VP              
     |        _________|___             
     |       |             NP          
     |       |     ________|____        
     |       |    |             PP     
     |       |    |         ____|___    
     NP      V    NP       P        NP 
     |       |    |        |        |   
Astronomers saw stars     with     ears

                      S                    
      ________________|____                 
     |                     VP              
     |            _________|________        
     |           VP                 PP     
     |        ___|____          ____|___    
     NP      V        NP  

In [7]:
def cyk_parse(sentence, grammar):
    n = len(sentence)
    table =np.empty((n, n), dtype=object) 
    for i, word in enumerate(sentence): 
        table[i, i] = [(p.lhs(), p.prob()) for p in grammar.productions() if len(p.rhs()) == 1 and p.rhs()[0] == word] 
    for span in range(2, n + 1): 
        for i in range(n - span + 1): 
            table[i, i + span - 1] = [(p.lhs(), lp * rp * p.prob()) for k in range(i, i + span - 1) 
                for l, lp in (table[i, k] or []) for r, rp in (table[k + 1, i + span - 1] or []) 
                for p in grammar.productions() if len(p.rhs()) == 2 and (l, r) == p.rhs()] 
    return table
table = cyk_parse(sentence,cnf_grammar)
r,p=table[0][2][0]
if str(r)=='S': print("Accepted -",p)
else: print("Not Accepted -",p)

Accepted - 0.0126


In [8]:
from nltk import PCFG, ViterbiParser 
viterbi_parser = ViterbiParser(cnf_grammar) 
vparses = list(viterbi_parser.parse(sentence)) 
vparse_data = [(str(tree), tree.prob()) for tree in vparses] 
vdf = pd.DataFrame(vparse_data, columns=['Parse Tree', 'Probability']) 
print("\nParse Results using Viterbi Parsing:\n",vdf)  
vmean_prob = np.mean(vdf['Probability'])
print("Mean Probability of Parses:", vmean_prob)


Parse Results using Viterbi Parsing:
                                           Parse Tree  Probability
0  (S\n  (NP Astronomers)\n  (VP (V saw) (NP (NP ...     0.000907
Mean Probability of Parses: 0.0009071999999999999
